Intro) A une époque ou la mahorité des réservations d"hôtels se font via des sites tiers tels que Booking.com, l'e-reputation est devenu un élément de préoccupation majeur pour les hôtels.Être capable d'analyser et d'extraire des insights à partir des commentaires en ligne (Note client moyenne, comparaison par rapport aux concurrents, mentions positivies/negatives...) est donc un atout pour les opérateurs d'hôtels.

Le but de ce projet est donc de réaliser une brève analyse NLP de 100k commentaires booking laissé sur les établissements du département des Pyrénnées-Atlantiques (64) entre 2019 et 2022.

Dans ce notebook, on lance un crawl via selenium pour recuperer les urls booking des établissements des Pyréennées Atlantiques. Ces urls seront ensuite utilisées par le fichier 2- Crawl Avis Booking pour récupérér les avis de chaque établissement.

Etant donné que Booking.com n'affiche que 1000 établissements en même maximum, et que sur le départment 64 il 
y en a 2500+ (https://www.booking.com/region/fr/pyrenees-atlantiques.fr.html), on va filtrer au fur et à mesure
les résultats en utilisant les filtres : étoiles, types d'établissements et quartiers pour diminuer le nombre de résultats.
Cela permettra d'obtenir l'ensemble des +2500 établissements

In [ ]:
import re
import time
import warnings
import concurrent.futures

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent

from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import Future
from concurrent.futures import ThreadPoolExecutor, as_completed

#Ignore deprectation & SSL certificate errors
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [ ]:
#Define subfunction to get all accomodation urls on a page
def urlfetch():
    """
    Fonction permettant de récupérer les urls des établissements
    affichés sur une page de résultats booking.
    Arguments:
        Pas d'arguments
        
    Renvoit: Null
        Ecrit sur le fichier de résultat les urls des établissements
    """
    lecture_urls = chrome.find_elements(By.CSS_SELECTOR,"a[class='e13098a59f']")
    hotels_urls= [url.get_attribute("href").split("?")[0] for url in lecture_urls]
    for urlhotel in hotels_urls:
        with open("booking_url.txt",'a',encoding ="utf-8") as file:
            file.write(urlhotel)
            file.write('\n')

In [ ]:
# Define main function
def searchcityurl(booking_url):
    """
    Fonction prenant en paramètre un url de ville, de région booking
    et crawlant l'ensemble des établissements listés sur cette destination.
    
    Afin de contourner la limite de 1000 résultats affichés en même temps,
    la function va, si nécessaire, cliquer sur les boutons de filtre booking pour 
    diminuer le nombre de résultats affichés afin de récupérer l'ensemble des 
    établissements segments par segments.
    
    Arguments:
        booking_url : String : Un url de destination booking.
        Par exemple : "https://www.booking.com/region/fr/pyrenees-atlantiques.fr.html"
        
    Renvoit: Null
        Crawl et ecrit sur le fichier de résultat les urls des établissements
        listés sur l'url de destination booking
    """
    try :
        for counter in range(5):
            try:
                #Driver
                chrome_options = Options()
                ua = UserAgent()
                userAgent = ua.random
                chrome_options.add_argument(f"user-agent={userAgent}")
                path = r"DRIVER_PATH"
                chrome = webdriver.Chrome(executable_path = path, options = chrome_options)
                chrome.maximize_window()
                
                #Go to booking url
                chrome.get(booking_url)
                time.sleep(2)
                break
            except:
                time.sleep(2)
                chrome.quit()
                '''
                PROCNAME = "chromedriver"
                for proc in psutil.process_iter():
                     if proc.name() == PROCNAME:
                          proc.kill()'''
        #Handle Popup
        time.sleep(1)
        pop_up=chrome.find_element(By.ID,'onetrust-reject-all-handler')
        try:
            pop_up.click()
            time.sleep(1)
        except:
            pass
        
        #Search hotels and accomodations
        time.sleep(1)
        search_button=chrome.find_element(By.CLASS_NAME,'sb-searchbox__button ')
        chrome.execute_script("arguments[0].scrollIntoView();", search_button)
        search_button.click()
        time.sleep(1)
        
        #Second Popup
        try:
            pop_up=chrome.find_element(By.ID,'onetrust-accept-all-handler')
            pop_up.click()
            time.sleep(1)
        except:
            pass
        
        #Get Number of accomodations in city/region
        element_=chrome.find_element(By.CSS_SELECTOR,"h1[class='e1f827110f d3a14d00da']")
        if len(element_)>0:
            element_=element.text
            element_=element_.replace(' ','').strip()
            try:
                element_=element_.replace('\xa0','').replace(' ','').strip()
            except:
                pass
            element_c=re.findall(r"(\d+)",element_)
            element_c_seuil=int(element_c[0])
        elif len(element_)==0:
            element_ = chrome.find_element(By.CSS_SELECTOR,"div[class='d8f77e681c']")
            element_=element.text
            element_=element_.replace(' ','').strip()
            try:
                element_=element_.replace('\xa0','').replace(' ','').strip()
            except:
                pass
            element_c=re.findall(r"(\d+)Etablissementstrouvés",element_)
            element_c_seuil=int(element_c[0])
        with open("Nombre_Etablissements_trouves.txt","a") as flog:
            print(element_c_seuil," etablissements sur l'url: ",x,file=flog)
        
        #Bouton pour naviguer entre les pages
            xpath_page_suivante = '//*[@id="search_results_table"]/div[2]/div/div/div/div[6]/div[2]/nav/div/div[3]/button'
        #1er cas : le nombre d'établissement <1000
        if element_c_seuil<=1000:
            #1ère page
            urlfetch()
            time.sleep(1)
            
            #Get all other pages
            lecture_toutes_pages=chrome.find_element(By.CSS_SELECTOR,"li[class='f32a99c8d1']")
            #Pages suivantes
            if len(lecture_toutes_pages)>0:
                max_pages=[]
                for page in lecture_toutes_pages:
                    try :
                        max_pages.append(int(re.findall(r'(\d+)',page.text)[0]))
                    except:
                        pass
                    max_page=max(max_pages)
                #Wait until button for next page is visible
                if max_page>1:
                    for counter_refresh in range(5):
                        try:
                            element = WebDriverWait(chrome, 4).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                        except:
                            try:
                                chrome.refresh()
                                time.sleep(3)
                                element = WebDriverWait(chrome, 4).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                            except:
                                continue
                    try:
                        #Click on next page button
                        click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                        chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                        chrome.execute_script("arguments[0].click();",click_element)
                        time.sleep(2)
                    except:
                        #Or Refresh and Click on next page button
                        chrome.refresh()
                        time.sleep(3)
                        click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                        chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                        chrome.execute_script("arguments[0].click();",click_element)
                        time.sleep(2)
                    #Loop on all the other pages
                    while True:
                        try:
                            #Keep 1st url to know if we reached the end of all the pages
                            url_0=str(chrome.current_url)
                            urlfetch()
                            time.sleep(1)
                            timeout = time.time() + 45
                            try:
                                element = WebDriverWait(chrome, 4).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                            except:
                                try:
                                    chrome.refresh()
                                    element = WebDriverWait(chrome, 4).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                except:
                                    break
                            try:
                                time.sleep(3)
                                click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                chrome.execute_script("arguments[0].click();",click_element)
                                time.sleep(2)
                            except:
                                chrome.refresh()
                                time.sleep(3)
                                click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                chrome.execute_script("arguments[0].click();",click_element)
                                time.sleep(2)
                            url_1=str(chrome.current_url)
                        except:
                            urlfetch()
                            if len(chrome.find_elements(By.XPATH,xpath_page_suivante))>0:
                                raise Exception("Failed at pressing next-page button-Timeout")
                                break
                            else:
                                raise Exception("Failed at pressing next-page button-Button not present...Check for completion")
                                break

                        if url_0==url_1:
                            break
        #If more than 1000 accomodation listed, we start by filtering based on stars rating
        elif element_c_seuil>1000:
            listetoiles = []
            try :
                e1=chrome.find_element(By.CSS_SELECTOR,"div[data-filters-item='class:class=1']")
                se1 = e1.now()[0]
                de1={'label':se1.find_element(By.CSS_SELECTOR,"div[class='a1b3f50dcd b2fe1a41c3 a1f3ecff04 db7f07f643 d1764ea78b']").text.strip(),'count':int(se1.find("span",{"class":"d8eab2cf7f a414c2b280"}).text.strip()),'id':'//input[@name="class=1"]'}
                listetoiles.append(de1)
            except:
                pass
            try :
                e2=chrome.find_element(By.CSS_SELECTOR,"div[data-filters-item='class:class=2']")
                se2 = e2.now()[0]
                de1={'label':se2.find_element(By.CSS_SELECTOR,"div[class='a1b3f50dcd b2fe1a41c3 a1f3ecff04 db7f07f643 d1764ea78b']").text.strip(),'count':int(se1.find("span",{"class":"d8eab2cf7f a414c2b280"}).text.strip()),'id':'//input[@name="class=1"]'}
                listetoiles.append(de2)
            except:
                pass
            try:
                e3=chrome.find_element(By.CSS_SELECTOR,"div[data-filters-item='class:class=3']")
                se3 = e3.now()[0]
                de3={'label':se3.find_element(By.CSS_SELECTOR,"div[class='a1b3f50dcd b2fe1a41c3 a1f3ecff04 db7f07f643 d1764ea78b']").text.strip(),'count':int(se1.find("span",{"class":"d8eab2cf7f a414c2b280"}).text.strip()),'id':'//input[@name="class=1"]'}
                listetoiles.append(de3)
            except:
                pass
            try:
                e4=chrome.find_element(By.CSS_SELECTOR,"div[data-filters-item='class:class=4']")
                se4 = e4.now()[0]
                de4={'label':se4.find_element(By.CSS_SELECTOR,"div[class='a1b3f50dcd b2fe1a41c3 a1f3ecff04 db7f07f643 d1764ea78b']").text.strip(),'count':int(se1.find("span",{"class":"d8eab2cf7f a414c2b280"}).text.strip()),'id':'//input[@name="class=1"]'}
                listetoiles.append(de4)
            except:
                pass
            try:
                e5=chrome.find_element(By.CSS_SELECTOR,"div[data-filters-item='class:class=5']")
                se5 = e5.now()[0]
                de5={'label':se5.find_element(By.CSS_SELECTOR,"div[class='a1b3f50dcd b2fe1a41c3 a1f3ecff04 db7f07f643 d1764ea78b']").text.strip(),'count':int(se1.find("span",{"class":"d8eab2cf7f a414c2b280"}).text.strip()),'id':'//input[@name="class=1"]'}
                listetoiles.append(de5)
            except:
                pass
            try:
                e0=chrome.find_element(By.CSS_SELECTOR,"div[data-filters-item='class:class=0']")
                se0 = e5.now()[0]
                de0={'label':se0.find_element(By.CSS_SELECTOR,"div[class='a1b3f50dcd b2fe1a41c3 a1f3ecff04 db7f07f643 d1764ea78b']").text.strip(),'count':int(se1.find("span",{"class":"d8eab2cf7f a414c2b280"}).text.strip()),'id':'//input[@name="class=1"]'}
                listetoiles.append(de0)
            except:
                pass
            #Save current url so we can reset filters 
            base1=chrome.current_url
            #For each filter, we are gonna click ion the button filter,  check if there are less than 1000
            #elements. If it the case, we can get all data. If not, we use a second filter (type of accomodation
            #and neighboor) and so on.
            #After its done for one rating filter, we reset the url and move on to the next rating filter
            for z in listetoiles:
                chrome.get(base1)
                try :
                    time.sleep(2)
                    elem = chrome.find_element(By.XPATH,z['id'])
                    chrome.execute_script("arguments[0].scrollIntoView();", elem)
                    chrome.execute_script("arguments[0].click();", elem)
                    time.sleep(2)
                    if z['count']<=1000:
                        urlfetch()
                        time.sleep(1)
                        try:
                            element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                        except:
                            try:
                                chrome.refresh()
                                element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                            except:
                                continue
                        try:
                            time.sleep(2)
                            click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                            chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                            chrome.execute_script("arguments[0].click();",click_element)
                            time.sleep(2)
                        except:
                            print(traceback.format_exc())
                            chrome.refresh()
                            time.sleep(2)
                            click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                            chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                            chrome.execute_script("arguments[0].click();",click_element)
                            time.sleep(2)
                        while True:
                            try:
                                url_0=str(chrome.current_url)
                                urlfetch()
                                time.sleep(1)
                                timeout = time.time() + 45
                                try:
                                    element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                except:
                                    try:
                                        chrome.refresh()
                                        element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                    except:
                                        continue
                                try:
                                    time.sleep(2)
                                    click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                    chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                    chrome.execute_script("arguments[0].click();",click_element)
                                    time.sleep(2)
                                except:
                                    chrome.refresh()
                                    time.sleep(2)
                                    click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                    chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                    chrome.execute_script("arguments[0].click();",click_element)
                                    time.sleep(2)
                                #time.sleep(4)
                                url_1=str(chrome.current_url)
                            except:
                                urlfetch()
                                if len(chrome.find_elements(By.XPATH,xpath_page_suivante))>0:
                                    raise Exception("Failed at pressing next-page button-Timeout")
                                    break
                                else:
                                    raise Exception("Failed at pressing next-page button-Button not present...Check for completion")
                                    break
                            if url_0==url_1:
                                break
                    if z['count']>1000:
                        base2=chrome.current_url
                        #Liste deroulantes
                        btns = chrome.find_elements_by_css_selector('button[class="fc63351294 a168c6f285 d65b7d20ae a25b1d9e47"]')
                        for btn in btns :
                            try :
                                chrome.execute_script("arguments[0].scrollIntoView();", btn)
                                chrome.execute_script("arguments[0].click();", btn)
                            except:
                                pass
                        #If more than 1000 accomodation even after filtering by rating
                        #we filterin based on accomodation type
                        types= chrome.find_elements_by_css_selector('div[data-filters-item*="ht_id:ht_id="]')
                        types = [x for x in types if len(x.text) > 0 ]
                        typesh=[]
                        for type_ in types:
                            xpath_type ='//input[@name="{}"]'.format(type_.get_attribute("data-filters-item").split(":")[1])
                            case={'label':type_.text.strip().split("\n")[0],'count':int(type_.text.strip().split("\n")[1]),'id':xpath_type}
                            typesh.append(case)
                        for type_ in typesh:
                            chrome.get(base2)
                            time.sleep(2)
                            try :
                                elem_type = chrome.find_element(By.XPATH,type_['id'])
                                chrome.execute_script("arguments[0].scrollIntoView();", elem_type)
                                chrome.execute_script("arguments[0].click();", elem_type)
                                time.sleep(2)
                                if type_['count']<=1000:
                                    urlfetch()
                                    time.sleep(1)
                                    try:
                                        element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                    except:
                                        try:
                                            chrome.refresh()
                                            element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                        except:
                                            continue
                                    try:
                                        time.sleep(2)
                                        click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                        chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                        chrome.execute_script("arguments[0].click();", click_element)
                                        time.sleep(2)
                                    except:
                                        chrome.refresh()
                                        time.sleep(2)
                                        click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                        chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                        chrome.execute_script("arguments[0].click();", click_element)
                                        time.sleep(2)
                                    while True:
                                        try:
                                            url_0=str(chrome.current_url)
                                            urlfetch()
                                            time.sleep(1)
                                            timeout = time.time() + 45
                                            try:
                                                element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                            except:
                                                try:
                                                    chrome.refresh()
                                                    element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                                except:
                                                    continue
                                            try:
                                                time.sleep(2)
                                                click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                                chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                                chrome.execute_script("arguments[0].click();",click_element)
                                                time.sleep(2)
                                            except:
                                                print(traceback.format_exc())
                                                chrome.refresh()
                                                time.sleep(2)
                                                click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                                chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                                chrome.execute_script("arguments[0].click();",click_element)
                                                time.sleep(2)
                                            url_1=str(chrome.current_url)
                                        except:
                                            urlfetch()
                                            if len(chrome.find_elements(By.XPATH,xpath_page_suivante))>0:
                                                raise Exception("Failed at pressing next-page button-Timeout")
                                                break
                                            else:
                                                raise Exception("Failed at pressing next-page button-Button not present...Check for completion")
                                                break
                                        if url_0==url_1:
                                            break
                                if type_['count']>1000:
                                    base3=chrome.current_url
                                    #Liste deroulantes
                                    btns = chrome.find_elements_by_css_selector('button[class="fc63351294 a168c6f285 d65b7d20ae a25b1d9e47"]')
                                    for btn in btns :
                                        try :
                                            chrome.execute_script("arguments[0].scrollIntoView();", btn)
                                            chrome.execute_script("arguments[0].click();", btn)
                                        except:
                                            pass
                                    #If more than 1000 accomodation even after filtering by rating and by type
                                    #we filter based on location
                                    locations= chrome.find_elements_by_css_selector('div[data-filters-item*="di:di="]')
                                    locations = [x for x in locations if len(x.text) > 0 ]
                                    time.sleep(1)
                                    districts=[]
                                    for loc in locations:
                                        xpath_district ='//input[@name="{}"]'.format(loc.get_attribute("data-filters-item").split(":")[1])
                                        case2={'label':loc.text.strip().split("\n")[0],'count':int(loc.text.strip().split("\n")[1]),'id':xpath_district}
                                        districts.append(case2)
                                    for district in districts:
                                        #print(base3)
                                        chrome.get(base3)
                                        time.sleep(2)
                                        try :
                                            elem_district = chrome.find_element(By.XPATH,district['id'])
                                            chrome.execute_script("arguments[0].scrollIntoView();", elem_district)
                                            chrome.execute_script("arguments[0].click();", elem_district)
                                            time.sleep(2)
                                            if district['count']<=1000:
                                                try:
                                                    urlfetch()
                                                    time.sleep(1)
                                                    try:
                                                        element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                                    except:
                                                        try:
                                                            chrome.refresh()
                                                            element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                                        except:
                                                            continue
                                                    try:
                                                        time.sleep(2)
                                                        click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                                        chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                                        chrome.execute_script("arguments[0].click();",click_element)
                                                        time.sleep(2)
                                                    except:
                                                        print(traceback.format_exc())
                                                        chrome.refresh()
                                                        time.sleep(2)
                                                        click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                                        chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                                        chrome.execute_script("arguments[0].click();",click_element)
                                                        time.sleep(2)
                                                    while True:
                                                        try:
                                                            url_0=str(chrome.current_url)
                                                            urlfetch()
                                                            time.sleep(1)
                                                            timeout = time.time() + 45
                                                            try:
                                                                element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                                            except:
                                                                try:
                                                                    chrome.refresh()
                                                                    element = WebDriverWait(chrome, 20).until(EC.presence_of_element_located((By.XPATH,xpath_page_suivante)))
                                                                except:
                                                                    continue
                                                            try:
                                                                time.sleep(2)
                                                                click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                                                chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                                                chrome.execute_script("arguments[0].click();",click_element)
                                                                time.sleep(2)
                                                            except:
                                                                chrome.refresh()
                                                                time.sleep(2)
                                                                click_element=chrome.find_element(By.XPATH,xpath_page_suivante)
                                                                chrome.execute_script("arguments[0].scrollIntoView();", click_element)
                                                                chrome.execute_script("arguments[0].click();",click_element)
                                                                time.sleep(2)
                                                            url_1=str(chrome.current_url)
                                                        except:
                                                            urlfetch()
                                                            if len(chrome.find_elements(By.XPATH,xpath_page_suivante))>0:
                                                                raise Exception("Failed at pressing next-page button-Timeout")
                                                                break
                                                            else:
                                                                raise Exception("Failed at pressing next-page button-Button not present...Check for completion")
                                                                break
                                                        if url_0==url_1:
                                                            break
                                                except:
                                                    options=z['label']+'//'+type_['label']+'//'+district['label']
                                                    print('Problem with',x,'options:',options)
                                                    continue
                                            if district['count']>1000:
                                                continue
                                        except:
                                            pass
                            except:
                                pass
                except :
                    pass
        chrome.quit()
        
    except Exception as e:
        chrome.quit()
        with open("Failed_urls.txt","a") as flog:
            print('Did not complete:',x,traceback.format_exc(),file=flog)

#Set up Multithreading
with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
    future_to_url = {executor.submit(searchcityurl, url): url for url in url_a_city}
    for future in tqdm(concurrent.futures.as_completed(future_to_url),total=len(future_to_url)):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            with open("Failed_processes.txt","a") as flog:
                print('%r generated an exception: %s' % (url, traceback.format_exc()),file=flog)
        else:
            with open("Completed_urls.txt","a") as flog:
                print('%r page is completed' % url,file=flog)